# Project: Tidal transformation of M33 
# Evolution of the internal stellar structure of M33 


## Disk Profile Movie Maker

In [1]:
# Marco Barragan
# ASTR400B
# 
# This code makes plots for making a movie for the disk Profile
# This code intends to output png files in order to make a movie
# using ffmpeg

In [2]:
# import modules
import numpy as np
import astropy.units as u
from astropy.constants import G
import scipy.optimize as so

# import plotting modules
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline

# my modules
from ReadFile import Read
from CenterOfMass import CenterOfMass
from MassProfile import MassProfile

In [3]:
def find_confidence_interval(x, pdf, confidence_level):
    return pdf[pdf > x].sum() - confidence_level

In [4]:
def density_contour(xdata, ydata, nbins_x, nbins_y, ax=None, **contour_kwargs):
    """ Create a density contour plot.
    Parameters
    ----------
    xdata : numpy.ndarray
    ydata : numpy.ndarray
    nbins_x : int
        Number of bins along x dimension
    nbins_y : int
        Number of bins along y dimension
    ax : matplotlib.Axes (optional)
        If supplied, plot the contour to this axis. Otherwise, open a new figure
    contour_kwargs : dict
        kwargs to be passed to pyplot.contour()

    Example Usage
    -------------
     density_contour(x pos, y pos, contour res, contour res, axis, colors for contours)
     e.g.:
     density_contour(xD, yD, 80, 80, ax=ax, colors=['red','orange', 'yellow', 'orange', 'yellow'])

    """

    H, xedges, yedges = np.histogram2d(xdata, ydata, bins=(nbins_x,nbins_y), normed=True)
    x_bin_sizes = (xedges[1:] - xedges[:-1]).reshape((1,nbins_x))
    y_bin_sizes = (yedges[1:] - yedges[:-1]).reshape((nbins_y,1))

    pdf = (H*(x_bin_sizes*y_bin_sizes))

    X, Y = 0.5*(xedges[1:]+xedges[:-1]), 0.5*(yedges[1:]+yedges[:-1])
    Z = pdf.T
    fmt = {}

    ### Adjust Here #### 

    # Contour Levels Definitions
    zero_sigma = so.brentq(find_confidence_interval, 0., 1., args=(pdf, 0.50))
    one_sigma = so.brentq(find_confidence_interval, 0., 1., args=(pdf, 0.68))
    two_sigma = so.brentq(find_confidence_interval, 0., 1., args=(pdf, 0.95))
    three_sigma = so.brentq(find_confidence_interval, 0., 1., args=(pdf, 0.99))

    # You might need to add a few levels
    one_sigma1 = so.brentq(find_confidence_interval, 0., 1., args=(pdf, 0.80))
    one_sigma2 = so.brentq(find_confidence_interval, 0., 1., args=(pdf, 0.90))

    # Array of Contour levels. Adjust according to the above
    levels = [zero_sigma,one_sigma, one_sigma1, one_sigma2, two_sigma, three_sigma][::-1]

    # contour level labels  Adjust accoding to the above.
    strs = ['0.50','0.68','0.80','0.90','0.95', '0.99'][::-1]


    ###### 

    if ax == None:
        contour = plt.contour(X, Y, Z, levels=levels, origin="lower", **contour_kwargs)
        for l, s in zip(contour.levels, strs):
            fmt[l] = s
        plt.clabel(contour, contour.levels, inline=True, fmt=fmt, fontsize=12)

    else:
        contour = ax.contour(X, Y, Z, levels=levels, origin="lower", **contour_kwargs)
        for l, s in zip(contour.levels, strs):
            fmt[l] = s
        ax.clabel(contour, contour.levels, inline=True, fmt=fmt, fontsize=12)

    return contour

In [5]:
# a function that will rotate the position and velocity vectors
# so that the disk angular momentum is aligned with z axis. 

def RotateFrame(posI,velI):
    # input:  3D array of positions and velocities
    # returns: 3D array of rotated positions and velocities such that j is in z direction

    # compute the angular momentum
    L = np.sum(np.cross(posI,velI), axis=0)
    # normalize the vector
    L_norm = L/np.sqrt(np.sum(L**2))

    # Set up rotation matrix to map L_norm to z unit vector (disk in xy-plane)

    # z unit vector
    z_norm = np.array([0, 0, 1])

    # cross product between L and z
    vv = np.cross(L_norm, z_norm)
    s = np.sqrt(np.sum(vv**2))

    # dot product between L and z 
    c = np.dot(L_norm, z_norm)

    # rotation matrix
    I = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
    v_x = np.array([[0, -vv[2], vv[1]], [vv[2], 0, -vv[0]], [-vv[1], vv[0], 0]])
    R = I + v_x + np.dot(v_x, v_x)*(1 - c)/s**2

    # Rotate coordinate system
    pos = np.dot(R, posI.T).T
    vel = np.dot(R, velI.T).T

    return pos, vel

In [6]:
# Make plots to make a movie
for i in range(10):
    # Compose the data filename (be careful about the folder)
    # Add a string of the filenumber to "000"
    ilbl = '000' + str(i)
    # Remove all but the last 3 digits
    ilbl = ilbl[-3:]
    filename = "M33_" + ilbl + '.txt'
    
    # Make filename according to Collin's naming scheme for ffpmeg
    outfile = f'plot_{i:03}.png'
    
    # Create a COM of object for M31 Disk Using Code from Assignment 4
    COMD = CenterOfMass(filename,2)

    # Compute COM of M31 using disk particles
    COMP = COMD.COM_P(0.1)
    COMV = COMD.COM_V(COMP[0],COMP[1],COMP[2])

    # Initialize COM position and velocity
    # Determine positions of disk particles relative to COM 
    xD = COMD.x - COMP[0].value 
    yD = COMD.y - COMP[1].value 
    zD = COMD.z - COMP[2].value 

    # total magnitude
    rtot = np.sqrt(xD**2 + yD**2 + zD**2)

    # Determine velocities of disk particles relatiev to COM motion
    vxD = COMD.vx - COMV[0].value 
    vyD = COMD.vy - COMV[1].value 
    vzD = COMD.vz - COMV[2].value 

    # total velocity 
    vtot = np.sqrt(vxD**2 + vyD**2 + vzD**2)

    # Vectors for r and v 
    r = np.array([xD,yD,zD]).T # transposed 
    v = np.array([vxD,vyD,vzD]).T
    
    # Rotate vectors so that the disk angular momentum is aligned with z axis
    rn, vn = RotateFrame(r,v)
    
    
    # Make plots and save them ----------------------------------------------------------
    fig, axs = plt.subplots(2, 2)
    # plot the particle density for M31 
    # can modify bin number (bin =100 smoothest)
    axs[0,0].hist2d(rn[:,0], rn[:,1], bins=150, norm=LogNorm(), cmap='magma')

    # make the contour plot
    # x pos, y pos, contour res, contour res, axis, colors for contours.
    density_contour(rn[:,0], rn[:,1], 80, 80, ax=axs[0,0], colors=['red','white', 'white', 'white', 'white'])
     
    

    # Rotated M31 Disk - EDGE ON

    # plot the particle density for M31 (x,z)
    # can modify bin number (bin =100 smoothest)
    axs[0, 1].hist2d(rn[:,0], rn[:,2], bins=150, norm=LogNorm(), cmap='magma')
    
    
    # plot the particle density for M31 (y,z)
    # can modify bin number (bin =100 smoothest)
    axs[1, 0].hist2d(rn[:,0], rn[:,2], bins=150, norm=LogNorm(), cmap='magma')

    
    
    for ax in axs.flat:
        ax.set(xlabel='x-label', ylabel='y-label')

    # Hide x labels and tick labels for top plots and y ticks for right plots.
    for ax in axs.flat:
        ax.label_outer()

    
    plt.savefig(outfile)
    plt.close()
    
    print(i)
    
    '''fig = plt.figure()
    DPI = fig.get_dpi()
    fig.set_size_inches(1200/float(DPI),610.0/float(DPI))'''

0
1
2
3
4
5
6
7
8
9
